In [2]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import os

# from embrapa_api.preprocessing.preprocessors import BasePreprocessor
from embrapa_api.config import CSV_FILES_FOLDER
from embrapa_api.preprocessing.constants import EXPORTACAO_PATHS

In [3]:
EXPORTACAO_PATHS

{'Vinhos': {'url': 'http://vitibrasil.cnpuv.embrapa.br/download/ExpVinho.csv',
  'path': '/Users/mauricioaraujo/Eng_ML/FIAP/Tech Challenges/FIAP_Projeto_01/data/csv_files/exportacao/ExpVinho.csv'},
 'Sucos': {'url': 'http://vitibrasil.cnpuv.embrapa.br/download/ExpSuco.csv',
  'path': '/Users/mauricioaraujo/Eng_ML/FIAP/Tech Challenges/FIAP_Projeto_01/data/csv_files/exportacao/ExpSuco.csv'},
 'Espumantes': {'url': 'http://vitibrasil.cnpuv.embrapa.br/download/ExpEspumantes.csv',
  'path': '/Users/mauricioaraujo/Eng_ML/FIAP/Tech Challenges/FIAP_Projeto_01/data/csv_files/exportacao/ExpEspumantes.csv'},
 'Uvas Frescas': {'url': 'http://vitibrasil.cnpuv.embrapa.br/download/ExpUva.csv',
  'path': '/Users/mauricioaraujo/Eng_ML/FIAP/Tech Challenges/FIAP_Projeto_01/data/csv_files/exportacao/ExpUva.csv'}}

In [4]:
exportacao_paths = EXPORTACAO_PATHS

In [5]:
exportacao_paths

{'Vinhos': {'url': 'http://vitibrasil.cnpuv.embrapa.br/download/ExpVinho.csv',
  'path': '/Users/mauricioaraujo/Eng_ML/FIAP/Tech Challenges/FIAP_Projeto_01/data/csv_files/exportacao/ExpVinho.csv'},
 'Sucos': {'url': 'http://vitibrasil.cnpuv.embrapa.br/download/ExpSuco.csv',
  'path': '/Users/mauricioaraujo/Eng_ML/FIAP/Tech Challenges/FIAP_Projeto_01/data/csv_files/exportacao/ExpSuco.csv'},
 'Espumantes': {'url': 'http://vitibrasil.cnpuv.embrapa.br/download/ExpEspumantes.csv',
  'path': '/Users/mauricioaraujo/Eng_ML/FIAP/Tech Challenges/FIAP_Projeto_01/data/csv_files/exportacao/ExpEspumantes.csv'},
 'Uvas Frescas': {'url': 'http://vitibrasil.cnpuv.embrapa.br/download/ExpUva.csv',
  'path': '/Users/mauricioaraujo/Eng_ML/FIAP/Tech Challenges/FIAP_Projeto_01/data/csv_files/exportacao/ExpUva.csv'}}

In [8]:
produto_importacao = "Vinhos"

try:
    data = pd.read_csv(exportacao_paths[produto_importacao]["url"], sep=';')
    
except Exception as e:
    print(e)
    data = pd.read_csv(exportacao_paths[produto_importacao]["path"], sep=';')


In [9]:
data.head()

,Id,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1,2023,2023.1
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,11,46,0,0,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,26,95,4,21,0,0,0,0,117,698
2,3,"Alemanha, República Democrática",0,0,0,0,4168,2630,12000,8250,...,3660,25467,6261,32605,2698,6741,7630,45367,4806,31853
3,4,Angola,0,0,0,0,0,0,0,0,...,345,1065,0,0,0,0,4068,4761,0,0
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
data.head()

,Id,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1,2023,2023.1
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,11,46,0,0,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,26,95,4,21,0,0,0,0,117,698
2,3,"Alemanha, República Democrática",0,0,0,0,4168,2630,12000,8250,...,3660,25467,6261,32605,2698,6741,7630,45367,4806,31853
3,4,Angola,0,0,0,0,0,0,0,0,...,345,1065,0,0,0,0,4068,4761,0,0
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
keys = ["Id", "País"]
valor_cols = [col for col in data.columns if '.1' in col]
qtd_importada_cols = [col for col in data.columns.difference(keys) if col not in valor_cols]


In [15]:
qtd_importadas_df = (
    data[keys + qtd_importada_cols]
    .melt(id_vars=keys, var_name="ano", value_name="QTD_EXPORTADO_KG")
    .rename(columns={
        "Id": "CD_PAIS",
        "País": "NM_PAIS",
        "ano": "DT_ANO",
    })
)
vr_valor_df = (
    data[keys + valor_cols]
    .melt(id_vars=keys, var_name="ano", value_name="VL_VALOR_EXPORTADO_USD")
    .rename(columns={
        "Id": "CD_PAIS",
        "País": "NM_PAIS",
        "ano": "DT_ANO",
    })
    .assign(DT_ANO= lambda x: x["DT_ANO"].str.split(".").str[0])
)


rf_data = (
    qtd_importadas_df
    .merge(vr_valor_df, on=["CD_PAIS", "NM_PAIS", "DT_ANO"])
    .assign(NM_ITEM=produto_importacao)
)
rf_data = (
    rf_data[
        ['NM_PAIS', 'DT_ANO', 'NM_ITEM', 'QTD_EXPORTADO_KG', 'VL_VALOR_EXPORTADO_USD']
    ]
    .sort_values(['NM_PAIS', 'DT_ANO'])
)


In [17]:
rf_data.tail()

,NM_PAIS,DT_ANO,NM_ITEM,QTD_EXPORTADO_KG,VL_VALOR_EXPORTADO_USD
6724,Áustria,2019,Vinhos,0,0
6861,Áustria,2020,Vinhos,0,0
6998,Áustria,2021,Vinhos,0,0
7135,Áustria,2022,Vinhos,6,212
7272,Áustria,2023,Vinhos,0,0


In [19]:
rf_data.query("NM_PAIS == 'Brasil'")

,NM_PAIS,DT_ANO,NM_ITEM,QTD_EXPORTADO_KG,VL_VALOR_EXPORTADO_USD
22,Brasil,1970,Vinhos,0,0
159,Brasil,1971,Vinhos,0,0
296,Brasil,1972,Vinhos,0,0
433,Brasil,1973,Vinhos,0,0
570,Brasil,1974,Vinhos,0,0
707,Brasil,1975,Vinhos,0,0
844,Brasil,1976,Vinhos,0,0
981,Brasil,1977,Vinhos,0,0
1118,Brasil,1978,Vinhos,0,0
1255,Brasil,1979,Vinhos,0,0


In [54]:
rf_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3672 entries, 0 to 3671
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   CD_PAIS                 3672 non-null   int64  
 1   NM_PAIS                 3672 non-null   object 
 2   DT_ANO                  3672 non-null   object 
 3   NM_ITEM                 3672 non-null   object 
 4   QTD_IMPORTADO_KG        3672 non-null   int64  
 5   VL_VALOR_IMPORTADO_USD  3671 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 200.8+ KB


# Teste classe

In [3]:
from embrapa_api.preprocessing.preprocessors import ImportacaoPreprocessor

In [6]:
importacaoPreprocessor = ImportacaoPreprocessor()

In [4]:
# vinhos = self.processa_vinhos()
# sucos = self.processa_sucos()
# espumantes = self.processa_espumantes()
# frescas = self.processa_frescas()
# passas = self.processa_passas()

# vinhos = importacaoPreprocessor.processa_vinhos()

In [7]:
importacao_df = importacaoPreprocessor.preprocess()

Failed to load data from URL. Loading from local file.
[Errno 54] Connection reset by peer


In [8]:
importacao_df.head()

,NM_PAIS,DT_ANO,NM_ITEM,QTD_IMPORTADO_KG,VL_VALOR_IMPORTADO_USD
8748,Afeganistão,1970,Passas,0,0.0
8749,Afeganistão,1971,Passas,0,0.0
8750,Afeganistão,1972,Passas,0,0.0
8751,Afeganistão,1973,Passas,0,0.0
8752,Afeganistão,1974,Passas,0,0.0


In [9]:
importacao_df.groupby("NM_PAIS")["DT_ANO"].nunique()

NM_PAIS
Afeganistão                        54
Africa do Sul                      54
Alemanha                           54
Alemanha, República Democrática    54
Argentina                          54
                                   ..
Venezuela                          54
itália                             54
África do Sul                      54
Áustria                            54
Índia                              54
Name: DT_ANO, Length: 94, dtype: int64